In [6]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.cm as cm

from math import sin,cos,acos,sqrt,pi, atan2

import pandas as pd

%matplotlib

Using matplotlib backend: TkAgg


In [744]:
?Timer

Init signature:
Timer(
    name: Union[str, NoneType] = None,
    text: str = 'Elapsed time: {:0.4f} seconds',
    logger: Union[Callable[[str], NoneType], NoneType] = <built-in function print>,
) -> None
Docstring:      Time your code using a class, context manager, or decorator
Type:           type
Subclasses:     


In [3]:
# timer.py

from contextlib import ContextDecorator
from dataclasses import dataclass, field
import time
from typing import Any, Callable, ClassVar, Dict, Optional

class TimerError(Exception):
    """A custom exception used to report errors in use of Timer class"""

@dataclass
class Timer(ContextDecorator):
    """Time your code using a class, context manager, or decorator"""

    timers: ClassVar[Dict[str, float]] = {}
    counts: ClassVar[Dict[str, int]] = {}
    
    name: Optional[str] = None
    text: str = "Elapsed time: {:0.4f} seconds"
    logger: Optional[Callable[[str], None]] = print
    _start_time: Optional[float] = field(default=None, init=False, repr=False)

    def __post_init__(self) -> None:
        """Initialization: add timer to dict of timers"""
        if self.name:
            self.timers.setdefault(self.name, 0)
            self.counts.setdefault(self.name, 0)

    def start(self) -> None:
        """Start a new timer"""
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._start_time = time.perf_counter()

    def stop(self) -> float:
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")

        # Calculate elapsed time
        elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None

        # Report elapsed time
        if self.logger:
            self.logger(self.text.format(elapsed_time))
        if self.name:
            self.timers[self.name] += elapsed_time
            self.counts[self.name] += 1

        return elapsed_time

    def __enter__(self) -> "Timer":
        """Start a new timer as a context manager"""
        self.start()
        return self

    def __exit__(self, *exc_info: Any) -> None:
        """Stop the context manager timer"""
        self.stop()


In [4]:
class isocapix:
    def __init__(self,n,fact=0,tol=10**(-6)):
        '''
            n: order of the pixelization,
                # of pixels = 10*n^2+2 
                among them 12 are pentagon-like  and the rest are hexagon-like
            tol: a rounding factor to distinguish between 2 floats
        '''
        self.n    = n
        self.tol  = tol
        self.fact = fact
        
        self.xoff7 = -1/2
        self.yoff7 = 0

        #icosedron vertices
        self.icoPoints = self.getIcosaedreVertices()
        
        # number of isocaedron faces
        self.nIsofaces = 20
        # All triplet vertices of the 20  equilateral triangles
        self.icoTriangs = [(0,i+1,(i+1)%5+1) for i in range(5)] +\
             [(6,i+7,(i+1)%5+7) for i in range(5)] +\
             [(i+1,(i+1)%5+1,(7-i)%5+7) for i in range(5)] +\
             [(i+1,(7-i)%5+7,(8-i)%5+7) for i in range(5)]
        self.icoTriangs=np.array(self.icoTriangs)
        
        
        # scaling factor   
        self.scale = 1/(n*sqrt(3))
        
        # build the different pixels
        Timer.timers={}
        Timer.counts={}
        
        self.t1 = Timer("Hexa All", logger=None)
        self.tmapGridpoint2Sogere = Timer("mapGridpoint2Sogere",logger=None) 
        self.tslerp = Timer("slerp", logger=None)
        self.tbarycentricCoords = Timer("barycentricCoords", logger=None)
        
        self.t1.start()
        self.dfhexa = self.makeHexagon()
        self.t1.stop()


        totHexa_time = Timer.timers["Hexa All"]

        print(f"Hexa: {totHexa_time:0.2f} seconds")

        totHexa_mapGridpoint2Sogere = Timer.timers["mapGridpoint2Sogere"]
        print(f"==> mapGridpoint2Sogere: {totHexa_mapGridpoint2Sogere:0.2f} sec")
        
        totHexa_slerp = Timer.timers["slerp"]
        totHexa_slerp_cnt = Timer.counts["slerp"]
        print(f"==> slerp: {totHexa_slerp:0.2f} sec, cnt: {totHexa_slerp_cnt}")

        totHexa_barycentricCoords = Timer.timers["barycentricCoords"]
        print(f"==> barycentricCoords: {totHexa_barycentricCoords:0.2f} sec")
        self.t2 = Timer("Penta All", logger=None)
        self.t2.start()
        self.dfpenta  = self.makePentagon()
        self.t2.stop()
        totPenta_time = Timer.timers["Penta All"]
        print(f"Penta: {totPenta_time:0.2f} seconds")

        
        
        
        self.dfAll = pd.concat([self.dfpenta, self.dfhexa], ignore_index=True)


    #################################################################
    def barycentricCoords(self,p):
        '''
            input: 'p'is are the position vector of the form [x,y]'. 
            output: l1,l2,l3 are the barycentric co-ordsinates.

            ex:
            barycentricCoords([1,2])
            (-1.6547005383792517, 0.3452994616207483, 2.3094010767585034)

            D'une maniere generale
            p1=(x1,y1), p2=(x2,y2) p3=(x3,y3)
            T= [[x1-x3,x2-x3],[y1-y3,y2-y3]]
            (l1,l2) = T^(-1) . ( (x,y)-p3 )
            l3 = 1-l2-l3
        '''
        self.tbarycentricCoords.start()

        x,y = p[0],p[1]
        # l3*sqrt(3)/2 = y
        l3 = y*2./sqrt(3.)
        # l1 + l2 + l3 = 1
        # 0.5*(l2 - l1) = x
        l2 = x + 0.5*(1 - l3)
        l1 = 1 - l2 - l3
        
        self.tbarycentricCoords.stop()

        
        return l1,l2,l3

    #################################################################
    def scalProd(self, p1,p2):
        '''
            input: p1 and p2 are the vetors of form [x0,x1,...,xn]'
            output: is the scalar product of p1 and p2.
            (nb. apply for list &  tuple & np.array)
        '''
        return sum([p1[i]*p2[i] for i in range(len(p1))])

    #################################################################
    def slerp(self,p0,p1,t):
        '''
            program outputs the spherical linear interpolation 
            of arc defined by p0, p1(around origin).  

            input: t=0 -> p0, t=1 -> p1. 
                    p0 and p1 are the vetors of form [x,y,z]

            output: interpolated coordinates.

            https://en.wikipedia.org/wiki/Slerp

        '''
###        assert abs(self.scalProd(p0,p0) - self.scalProd(p1,p1)) < self.tol

        self.tslerp.start()
    
        ang0Cos = self.scalProd(p0,p1)/self.scalProd(p0,p0)
        ang0Sin = sqrt(1 - ang0Cos*ang0Cos)
        ang0 = atan2(ang0Sin,ang0Cos)
        l0 = sin((1-t)*ang0)
        l1 = sin(t    *ang0)
        tmp= np.array([(l0*p0[i] + l1*p1[i])/ang0Sin for i in range(len(p0))])
        
        self.tslerp.stop()
        return tmp
    
    #################################################################
    def mapGridpoint2Sogere(self,p,s1,s2,s3):
        '''
            program outputs the coordinate array of the projection of the input
            coordinates on the unit sphere.   
            inputs:
                - 'p' is the coordinate array of the planer verticies of the closed 
                    shape to be projected in the form [x,y,z]'.
                - 's1','s2' and 's3' are the vectors defining plane of the co-ordinates 
                    to be projected. 
            output: is the coordinate array of the projected face on the unit sphere.

            ex. mapGidpoint2Sogere([0,0.5,0.5],[1,0,0]',[0,1,0]',[0,0,1]')
        '''
                    
        self.tmapGridpoint2Sogere.start() 

        l1,l2,l3 = self.barycentricCoords(p)
        if abs(l3-1) < self.tol: return s3
        l2s = l2/(l1+l2)
        p12 = self.slerp(s1,s2,l2s)
        tmp= self.slerp(p12,s3,l3)
        
        self.tmapGridpoint2Sogere.stop() 
        return tmp

    #################################################################
    def hexagon(self,x,y,th,opt):
        '''
            this program creates the hexagon of given configuration and size.
            inputs: 
                - x and y are the rectangular coordinates of the center of the hexagon
                - th is the rotation angle measured anticlockwise positive 
                    from positive x axis
                - scale : is a contraction/dilation factor
                - opt: 1 full hexagon. 2 half hexagon
                - fact: a factor to shrink the hexagons neightbooring the pentagons  

                output: planar hexagon (complete/truncated) orientated 
            example:
                hexagon(0,0,np.pi/6,0.5,1)
        '''
        # rotation matrx with scale (th>0 the transformation is anti-clockwise)
        rot_mat = self.scale * np.array([[np.cos(th), -np.sin(th)],
                        [np.sin(th), np.cos(th)]])
        if opt == 1:
            '''
                Hexagone complet
                                       Y
                      0                ^
                1           5          I
                                       I--- > X
                2           4
                      3     
            ''' 
            hex = np.zeros((2,6))
            hex[0,:]= np.array([np.sin(i*np.pi/3) for i in range(6)]) # X-coord
            hex[1,:]= np.array([np.cos(i*np.pi/3) for i in range(6)]) # Y-coord

        elif opt == 2:
            '''
                Hexagone tronque

                        2               ^
                 3             1         I
                                         I
                 4             0         I--- >
            ''' 
            hex = np.zeros((2,5))
            hex[0,:]= np.array([sqrt(3)/2,sqrt(3)/2,0,-sqrt(3)/2,-sqrt(3)/2]) # X-ccod
            hex[1,:]= np.array([0,0.5,1,0.5,0]) # Y-coord

        elif opt == 3:
            # point 0 et 1 sont modifiers par rapport au type 2
            hex = np.zeros((2,5))
            hex[0,:]= np.array([sqrt(3)/2-self.fact,sqrt(3)/2-self.fact,0,-sqrt(3)/2,-sqrt(3)/2]) # X-ccod
            hex[1,:]= np.array([0,1/2+self.fact/sqrt(3),1,0.5,0]) # Y-coord

        elif opt == 4:
            # point 3 et 4 sont modifiers par rapport au type 2
            hex = np.zeros((2,5))
            hex[0,:]= np.array([sqrt(3)/2,sqrt(3)/2,0,-sqrt(3)/2+self.fact,-sqrt(3)/2+self.fact]) # X-ccod
            hex[1,:]= np.array([0,0.5,1,1/2+self.fact/sqrt(3),0]) # Y-coord


        hex = np.matmul(rot_mat,hex)

        hex[0,:]= x+hex[0,:] 
        hex[1,:]= y+hex[1,:]

        return hex
    
    #################################################################
    def getIcosaedreVertices(self):
        """
            outputs location of the icosaedre vertices 3D points
        """
        #golden ratio
        phi = 0.5*(1+sqrt(5)) 

        topPoints = \
            [(phi,1,0)]+\
            [(phi,-1,0)]+\
            [(1,0,-phi)]+\
            [(0,phi,-1)]+\
            [(0,phi,1)]+\
            [(1,0,phi)]

        topPoints = np.array(topPoints)
        # rot clockwise arround Z pour amener le point 1 en position (1,0,0)
        sinth = 1/sqrt(1+phi**2)
        costh = phi*sinth
        scale = 1/sqrt(1+phi**2)
        rot_mat = scale*np.array([[costh,sinth,0],
                            [-sinth, costh,0],
                            [0,0,1]])

        for i in range(len(topPoints)):
            topPoints[i,:] = np.matmul(rot_mat,topPoints[i,:])

        # change de repere
        # X' = -Y, Y'=-Z, Z'=X
        tmp = np.zeros_like(topPoints)
        for i in range(topPoints.shape[0]):
            tmp[i,0] = -topPoints[i,1]
            tmp[i,1] = -topPoints[i,2]
            tmp[i,2] =  topPoints[i,0]
        topPoints = tmp

        # points du bas de l'icosaedre
        bottomPoints = np.zeros_like(topPoints)
        for i in range(bottomPoints.shape[0]):
            bottomPoints[i,0] = -topPoints[i,0]
            bottomPoints[i,1] =  topPoints[i,1]
            bottomPoints[i,2] = -topPoints[i,2]

        # icosaedre vertices
        icoPoints=np.vstack((topPoints,bottomPoints))

        #return
        return icoPoints
    
    #################################################################
    def getProjectedFace(self,hexag,u,v,w):
        """
            outputs the coordinates of projected face on the plane 
            defined by tips of the vectors u,v and w on the unit radius sphere.  

             Inputs:

                _ 'hexag' is the coordinate array of the planer verticies of the closed 
                 shape to be projected in the form [x,y,z]'.
                _ 'u','v' and 'w' are the vectors defining plane to be projected on 
                     the sphere.
                _ 'icoPoints': icosedre 3D vertices

        """
    
        n = hexag.shape[1]
        face = np.zeros((3,n))
        # projecting the input hexagonal mesh on the sphere
        for i in range(n):

            face[:,i] = self.mapGridpoint2Sogere(hexag[:,i],
                                            self.icoPoints[u,:],
                                            self.icoPoints[v,:],
                                            self.icoPoints[w,:])
    
        return face

    #################################################################
    def getProjectedPt(self,p,u,v,w):
        """
        p: 2D point location
        """
        return self.mapGridpoint2Sogere(p,
                                    self.icoPoints[u,:],
                                    self.icoPoints[v,:],
                                    self.icoPoints[w,:])
    
    #################################################################
    def rounding(self,x):
        """
            transform "x" (eg. list of floats) to tuple of integers up to a certain precision 
        """
        return tuple(np.round(np.array(x)/self.tol).astype(int))

    #################################################################
    def tofloat(self,x):
        """
            reverse of 'rounding' method up to the precision level
        """
        return tuple(np.array(x)*self.tol)

    #################################################################
    def keep_unique(self,x):
        a=[i for i in x if x.count(i)==1]
        return a
        
        
    #################################################################
    def cmpdist(self,x):
        """
            Context: After glueing half-hexagonal pixels the chain of the vertices
          x=(p0,p1,p2,p3,p4,p5) can be in a wrong sequence. 
           So this function compare dist(p2,p3) versus dist(p2,p5) to trigger a swapping operation 
           (see next function)
        """
        pt2,pt3,pt5=np.array(x[2]),np.array(x[3]),np.array(x[5])
        return ((pt2-pt3)**2).sum()<((pt2-pt5)**2).sum()
    
    #################################################################
    def swapt(self,x):
        x[3],x[5]=x[5],x[3]
        return x

    #################################################################
    # Main method to build hexagons
    #################################################################
    def makeHexagon(self):
        """
        return a dataframe with the hexagones
        """
        tmakeHexagon = Timer("makeHexagon", logger=None)
        tmakeHexagon.start()
        
        icoTriangs = self.icoTriangs
        faces = self.nIsofaces
        n = self.n
        
        listDF=[]
            
        #### df = pd.DataFrame(columns=['idx','type','center','vertices'])

        
        #exhaustive loop over the 20 faces of the icosaedre to build each hexagon
        thexa = Timer("hexagon", logger=None)
        tprojFace = Timer("getProjectedFace", logger=None)
        tprojPt   = Timer("getProjectedPt",logger=None)
        tdfappend = Timer("dfappend",logger=None)
        for k in range(faces):
                # (i,j) is a couple of indice that uniquely identify an hexagon
                # opt=1: full hexagone in the "middle" of the face
                # opt=2: hexagone at a edge between two faces but not affected by a possible deformation 
                # opt=3 or 4: hexagone like opt=2 but which neightboor of a pentagon so possibly
                #             affected by a deformation
            for i in range(n+1):
                for j in range(n-i+1):
                    if i==0:
                        th = -2*pi/3
                        if j==1:
                            opt = 3
                        elif j==n-i-1:
                            opt = 4
                        else:
                            opt = 2
                    elif j==n-i: 
                        th = 2*pi/3
                        if i==1:
                            opt = 3
                        elif i==n-1:
                            opt = 4
                        else:
                            opt = 2
                    elif (j==0 and i != 0):
                        th = 0
                        if i==1:
                            opt = 4
                        elif i==n-1:
                            opt = 3
                        else:
                            opt = 2
                    else:
                        opt = 1
                        th = 0
                    #exclude the hexagons at the vertices of the isocele triangle
                    if (i!=0 or j!=0) and (i!=0 or j!=n) and (i!=n or j!=0):
                        #make the hexagon in the 2D frame of a generic equlateral trinagle
                        # orig hexagcenter = np.array([self.xoff7+i*1/n+j*1/(2*n), self.yoff7+j*sqrt(3)/(2*n)])
                        hexagcenter = np.array(self.xycent(self.xoff7,self.yoff7,i,j,n))
                        thexa.start()
                        hexag = self.hexagon(hexagcenter[0],hexagcenter[1],th,opt)
                        thexa.stop()
                        #project it on the sphere
                        a = self.icoTriangs[k,0]
                        b = self.icoTriangs[k,1]
                        c = self.icoTriangs[k,2]
                        #get list of the 3D vertices of the hexagon (possibly truncated and deformed)
                        tprojFace.start()
                        face = self.getProjectedFace(hexag,a,b,c)
                        tprojFace.stop()
                        xf,yf,zf = face[0,:],face[1,:],face[2,:]
                        vertsf=list(zip(xf,yf,zf))
                        #get the 3D position of the original center
                        #
                        # Todo: for deformed hexagon this should be changed later after the glue of half
                        #       hexagon shared by 2 faces
                        # 
                        tprojPt.start()
                        center = self.getProjectedPt(hexagcenter,a,b,c)
                        tprojPt.stop()
                        # Dataframe update
                        tdfappend.start()
                        listDF.append([(k,i,j),opt,(center[0],center[1],center[2]),vertsf])
                         
#                        df=df.append({'idx':(k,i,j),
#                                             'type':opt,
#                                              'center':(center[0],center[1],center[2]),
#                                              'vertices':vertsf},ignore_index=True)
                        tdfappend.stop()
                        
        
        #eo loop on faces
        
        tdfOperations= Timer("dfOpAll",logger=None)
        tdfOperations.start()

        df = pd.DataFrame(listDF,columns=['idx','type','center','vertices'])
        
        # transform the 3D 'float' positions into 3D 'integer' position up to a precision
        #     to be able to perform groupby etc. Nb. otherwise groupby with tuple of floats cannot
        #     manage to merge for instance two 3D points close from each other due to precision
        df['center']  =df['center'].map(self.rounding)
        df['vertices']=df['vertices'].map(self.rounding)
        # groupby center of the pixel polygon 
        df=df.groupby('center', as_index=False).agg({'idx':'sum','type':lambda x: list(x),'vertices':'sum'})
        # for each pixel polygon keep only the vertices that occure once. By this operation we kill
        # vertices that are shared by half-hexagons of two faces that are glued to make a single hexagon
        # at icosaedre edges.
        df['vertices']=df['vertices'].map(lambda x: [list(y) for y in x])\
                        .map(lambda x: tuple([tuple(y) for y in x]))\
                        .map(self.keep_unique)
        # After glueing half-hexagonal pixels the chain of the vertices
         #x=(p0,p1,p2,p3,p4,p5) can be in a wrong sequence. 
        df['good']=df['vertices'].map(self.cmpdist)
        mask = (df['good'] == False)
        df_tbm = df[mask]
        df.loc[mask, 'vertices']= df_tbm['vertices'].map(self.swapt)
        #clean
        df=df.drop(['good'],axis=1)
        #re organize the columns which as been changed after the groupby center
        df=df[df.columns[[1,2,3,0]]]

        #rephrase the typology of the hexagons
        # 1: hexagons in the middle of each icosaedre faces
        # 2: hexagons at edges of two icosaedre faces
        # 3: hexagons neightbooring a pentagon
        df['type']=df['type'].map(lambda x: 1 if x==[1] else (2 if x==[2,2] else 3))
 
        #rescale to get floating numbers for the vertices and the center
        df['vertices']=df['vertices'].map(lambda x: np.array(x)*self.tol)
        df['center']=df['center'].map(lambda x: np.array(x)*self.tol)
        
                
        tdfOperations.stop()

        tmakeHexagon.stop()

        
        #got it
        tot_time = Timer.timers["hexagon"]
        print(f"hexagon: {tot_time:0.2f} seconds")
        
        tot_time = Timer.timers["getProjectedFace"]
        print(f"projFace: {tot_time:0.2f} seconds")
        
        tot_time  = Timer.timers["getProjectedPt"]
        print(f"projPt: {tot_time:0.2f} seconds")
        
        tot_time= Timer.timers["dfappend"]
        print(f"DfAppend: {tot_time:0.2f} seconds")

        tot_time = Timer.timers["dfOpAll"]
        print(f"Df processing All: {tot_time:0.2f} seconds")
    
        tot_time = Timer.timers["makeHexagon"]
        print(f"(verif) makeHexagon: {tot_time:0.2f} seconds")
        
        return df
    
    #################################################################
    def xycent(self,xoff7,yoff7,i,j,n):
        '''
            2D localisation of the center of a pentagon in the frame of a icosaedre face
        '''
        return xoff7+i/n+j/(2*n), yoff7+j*sqrt(3)/(2*n)

        
    #################################################################
    # Main method to build pentagons
    #################################################################
    def makePentagon(self):
        """
         There are 12 only pentagons
        """
        
        pentaBuild=pd.DataFrame(columns=['idx','face','xyc','th'])
        
        #below idx0 is a tuple with the icosaedre face number (the order is important)
        #we build a DF of the vertices of each pentagon positionned in local 2D icosadre-face frame
        
        xoff7 = self.xoff7
        yoff7 = self.yoff7
        n     = self.n
                        
        #Penta #0 : top
        idx0 = (0,1,2,3,4)
        for k in idx0:
            info = {
                'idx':idx0,
                'face':k,
                'xyc':self.xycent(xoff7,yoff7,1,0,n),
                'th':0 #'th':-2*pi/3
            }
            pentaBuild=pentaBuild.append(info,ignore_index=True)

        
        ######
        #Pentas of the upper ring
        ######
        #Penta #1 :
        idx0 = (0,1,11,16,10)
        infos=[]
        infos.append({
                'idx':idx0,
                'face':0,
                'xyc':self.xycent(xoff7,yoff7,0,n-1,n),
                'th':-2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':1,
                'xyc':self.xycent(xoff7,yoff7,n-1,1,n),
                'th':2*pi/3
            })

        infos.append({
                'idx':idx0,
                'face':11,
                'xyc':self.xycent(xoff7,yoff7,1,0,n),
                'th': 0
            })

        infos.append({
                'idx':idx0,
                'face':16,
                'xyc':self.xycent(xoff7,yoff7,1,0,n),
                'th': 0
            })

        infos.append({
                'idx':idx0,
                'face':10,
                'xyc':self.xycent(xoff7,yoff7,n-1,1,n),
                'th': 2*pi/3
            })

        for info in infos:
            pentaBuild=pentaBuild.append(info,ignore_index=True)

        #Penta #2
        idx0 = (1,2,12,17,11)
        infos=[]
        infos.append({
                'idx':idx0,
                'face':1,
                'xyc':self.xycent(xoff7,yoff7,0,n-1,n),
                'th':-2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':2,
                'xyc':self.xycent(xoff7,yoff7,n-1,1,n),
                'th':2*pi/3
            })

        infos.append({
                'idx':idx0,
                'face':12,
                'xyc':self.xycent(xoff7,yoff7,1,0,n),
                'th': 0
            })

        infos.append({
                'idx':idx0,
                'face':17,
                'xyc':self.xycent(xoff7,yoff7,1,0,n),
                'th': 0
            })

        infos.append({
                'idx':idx0,
                'face':11,
                'xyc':self.xycent(xoff7,yoff7,n-1,1,n),
                'th': 2*pi/3
            })

        for info in infos:
            pentaBuild = pentaBuild.append(info,ignore_index=True)

        #Penta #3 :
        idx0 = (2,3,13,18,12)
        infos=[]
        
        infos.append({
                'idx':idx0,
                'face':2,
                'xyc':self.xycent(xoff7,yoff7,0,n-1,n),
                'th':-2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':3,
                'xyc':self.xycent(xoff7,yoff7,n-1,1,n),
                'th':2*pi/3
            })

        infos.append({
                'idx':idx0,
                'face':13,
                'xyc':self.xycent(xoff7,yoff7,1,0,n),
                'th': 0
            })

        infos.append({
                'idx':idx0,
                'face':18,
                'xyc':self.xycent(xoff7,yoff7,1,0,n),
                'th': 0
            })

        infos.append({
                'idx':idx0,
                'face':12,
                'xyc':self.xycent(xoff7,yoff7,n-1,1,n),
                'th': 2*pi/3
            })

        
        for info in infos:
            pentaBuild = pentaBuild.append(info,ignore_index=True)

        
        #Penta #4 :
        idx0 = (3,4,14,19,13)
        infos=[]

        infos.append({
                'idx':idx0,
                'face':3,
                'xyc':self.xycent(xoff7,yoff7,0,n-1,n),
                'th':-2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':4,
                'xyc':self.xycent(xoff7,yoff7,n-1,1,n),
                'th':2*pi/3
            })

        infos.append({
                'idx':idx0,
                'face':14,
                'xyc':self.xycent(xoff7,yoff7,1,0,n),
                'th': 0
            })

        infos.append({
                'idx':idx0,
                'face':19,
                'xyc':self.xycent(xoff7,yoff7,1,0,n),
                'th': 0
            })

        infos.append({
                'idx':idx0,
                'face':13,
                'xyc':self.xycent(xoff7,yoff7,n-1,1,n),
                'th': 2*pi/3
            })
        
        for info in infos:
            pentaBuild = pentaBuild.append(info,ignore_index=True)

        #Penta #5 : 
        idx0 = (4,0,10,15,14)
        infos=[]

        infos.append({
                'idx':idx0,
                'face':4,
                'xyc':self.xycent(xoff7,yoff7,0,n-1,n),
                'th':-2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':0,
                'xyc':self.xycent(xoff7,yoff7,n-1,1,n),
                'th':2*pi/3
            })

        infos.append({
                'idx':idx0,
                'face':10,
                'xyc':self.xycent(xoff7,yoff7,1,0,n),
                'th': 0
            })

        infos.append({
                'idx':idx0,
                'face':15,
                'xyc':self.xycent(xoff7,yoff7,1,0,n),
                'th': 0
            })

        infos.append({
                'idx':idx0,
                'face':14,
                'xyc':self.xycent(xoff7,yoff7,n-1,1,n),
                'th': 2*pi/3
            })

        
        for info in infos:
            pentaBuild = pentaBuild.append(info,ignore_index=True)


        ######
        #Pentas of the lower ring
        ######
        """
        cases=np.array([
    [0,n-1,-2*pi/3,4],
    [n-1,1,2*pi/3,4],
    [1,0,0.,4]
])
        """

        #Penta #6 :
        idx0 = (6,7,15,10,16)
        infos=[]
        infos.append({
                'idx':idx0,
                'face':6,
                'xyc':self.xycent(xoff7,yoff7,0,n-1,n),
                'th':-2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':7,
                'xyc':self.xycent(xoff7,yoff7,n-1,1,n),
                'th': 2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':15,
                'xyc':self.xycent(xoff7,yoff7,n-1,1,n),
                'th': 2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':10,
                'xyc':self.xycent(xoff7,yoff7,0,n-1,n),
                'th': -2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':16,
                'xyc':self.xycent(xoff7,yoff7,0,n-1,n),
                'th': -2*pi/3
            })
        for info in infos:
            pentaBuild = pentaBuild.append(info,ignore_index=True)

        #Penta #7 :
        idx0 = (5,6,16,11,17)
        infos=[]

        infos.append({
                'idx':idx0,
                'face':6,
                'xyc':self.xycent(xoff7,yoff7,0,n-1,n),
                'th':-2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':7,
                'xyc':self.xycent(xoff7,yoff7,n-1,1,n),
                'th': 2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':15,
                'xyc':self.xycent(xoff7,yoff7,n-1,1,n),
                'th': 2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':10,
                'xyc':self.xycent(xoff7,yoff7,0,n-1,n),
                'th': -2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':16,
                'xyc':self.xycent(xoff7,yoff7,0,n-1,n),
                'th': -2*pi/3
            })
        for info in infos:
            pentaBuild = pentaBuild.append(info,ignore_index=True)


        #Penta #8 
        idx0 = (9,5,17,12,18)
        infos=[]

        infos.append({
                'idx':idx0,
                'face':6,
                'xyc':self.xycent(xoff7,yoff7,0,n-1,n),
                'th':-2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':7,
                'xyc':self.xycent(xoff7,yoff7,n-1,1,n),
                'th': 2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':15,
                'xyc':self.xycent(xoff7,yoff7,n-1,1,n),
                'th': 2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':10,
                'xyc':self.xycent(xoff7,yoff7,0,n-1,n),
                'th': -2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':16,
                'xyc':self.xycent(xoff7,yoff7,0,n-1,n),
                'th': -2*pi/3
            })
        
        
        for info in infos:
            pentaBuild = pentaBuild.append(info,ignore_index=True)

        #Penta #9 
        idx0 = (8,9,18,13,19)
        infos=[]
        infos.append({
                'idx':idx0,
                'face':6,
                'xyc':self.xycent(xoff7,yoff7,0,n-1,n),
                'th':-2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':7,
                'xyc':self.xycent(xoff7,yoff7,n-1,1,n),
                'th': 2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':15,
                'xyc':self.xycent(xoff7,yoff7,n-1,1,n),
                'th': 2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':10,
                'xyc':self.xycent(xoff7,yoff7,0,n-1,n),
                'th': -2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':16,
                'xyc':self.xycent(xoff7,yoff7,0,n-1,n),
                'th': -2*pi/3
            })
        
        
        
        for info in infos:
            pentaBuild = pentaBuild.append(info,ignore_index=True)


        
        #Penta #10
        idx0 = (7,8,19,14,15)
        infos=[]
        infos.append({
                'idx':idx0,
                'face':6,
                'xyc':self.xycent(xoff7,yoff7,0,n-1,n),
                'th':-2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':7,
                'xyc':self.xycent(xoff7,yoff7,n-1,1,n),
                'th': 2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':15,
                'xyc':self.xycent(xoff7,yoff7,n-1,1,n),
                'th': 2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':10,
                'xyc':self.xycent(xoff7,yoff7,0,n-1,n),
                'th': -2*pi/3
            })
        infos.append({
                'idx':idx0,
                'face':16,
                'xyc':self.xycent(xoff7,yoff7,0,n-1,n),
                'th': -2*pi/3
            })

        for info in infos:
            pentaBuild = pentaBuild.append(info,ignore_index=True)
        
        
        #Penta #11  : bottom
        idx0 = (5,6,7,8,9)
        for k in idx0:
            info = {
                'idx':idx0,
                'face':k,
                'xyc':self.xycent(xoff7,yoff7,1,0,n),
                'th':0 #-2*pi/3
            }
            pentaBuild=pentaBuild.append(info,ignore_index=True)

            
        #We can know drop face column
        pentaBuild = pentaBuild.drop('face',axis=1)
                
        #We group by idx and then get (x,y,th) triplet list
        pentaBuild=pentaBuild.groupby('idx',as_index=False).agg(lambda x: x.tolist())
        pentaBuild['info']=[[(*a, b) for a, b in zip(x, y)] for x, y in zip(pentaBuild['xyc'],pentaBuild['th'])]
    
        pentaBuild=pentaBuild.drop(['xyc','th'],axis=1)

        #print(pentaBuild.head(10))
        
        
        #get the 3D coordinates of the pentagon vertices
        for row in pentaBuild.itertuples():
            def make_pts3d(row):
                idx0 = row.idx
                info0 = np.array(row.info) 
                pts3d = []
                for ik,k in enumerate(idx0):
                    a = self.icoTriangs[k,0]
                    b = self.icoTriangs[k,1]
                    c = self.icoTriangs[k,2]
                    
                    #print(k,a,b,c)
                    
                    xc,yc,th=info0[ik][0],info0[ik][1],info0[ik][2]
                    ##pt2d = np.array([sqrt(3)/2-self.fact,1/2+self.fact/sqrt(3)]) # type 3 
                    pt2d = np.array([-sqrt(3)/2+self.fact,1/2+self.fact/sqrt(3)]) # type 4 
                    rot_mat = self.scale * np.array([[np.cos(th), -np.sin(th)],
                                                [np.sin(th), np.cos(th)]])
                    

                    pt2d = np.matmul(rot_mat,pt2d)
                    pt2d[0] += xc
                    pt2d[1] += yc


                    #pt3d = self.rounding(self.getProjectedPt(pt2d,a,b,c))
                    pt3d = self.getProjectedPt(pt2d,a,b,c)
                    
                    
                    pts3d.append(pt3d)

                pts3d = np.array(list(pts3d))#*self.tol
                vertsf=list(zip(pts3d[:,0],pts3d[:,1],pts3d[:,2]))
                return vertsf
        #
        pentaBuild['vertices']=pentaBuild.apply(make_pts3d, axis=1)
        #drop the intermediate "info"
        pentaBuild=pentaBuild.drop('info',axis=1)
        # compute pentagon barycenter and project onto the sphere
        pentaBuild['center']=pentaBuild['vertices']\
                    .map(lambda x: np.array(x).mean(axis=0))\
                    .map(lambda x: x/sqrt(sum(x*x)))
        
        #To uniformize with the DF of the hexagons
        pentaBuild['type']=0
        pentaBuild=pentaBuild[pentaBuild.columns[[0,3,1,2]]]
        
        
        
        # Got it!
        return pentaBuild
    


In [824]:
mypix=isocapix(10,fact=0)

hexagon: 0.08 seconds
projFace: 0.32 seconds
projPt: 0.07 seconds
DfAppend: 0.00 seconds
Df processing All: 1.70 seconds
(verif) makeHexagon: 2.25 seconds
Hexa: 2.25 seconds
==> mapGridpoint2Sogere: 0.35 sec
==> slerp: 0.26 sec, cnt: 16560
==> barycentricCoords: 0.03 sec


In [299]:
mypix.dfpenta

,idx,type,vertices,center
0,"(0, 1, 2, 3, 4)",0,"[(0.1233863543090565, 0.08663485764000903, 0.9...","[-5.334534269745011e-17, 5.615299231310538e-18..."
1,"(0, 1, 11, 16, 10)",0,"[(0.3478871122223193, 0.7497505146805842, 0.56...","[0.2777588761430904, 0.8482349175082626, 0.450..."
2,"(1, 2, 12, 17, 11)",0,"[(-0.6055520827818937, 0.5625459555918041, 0.5...","[-0.7208871325786141, 0.5262833938469894, 0.45..."
3,"(2, 3, 13, 18, 12)",0,"[(-0.7221388813398195, -0.40207799389106014, 0...","[-0.7232916261291257, -0.5229738923961758, 0.4..."
4,"(3, 4, 14, 19, 13)",0,"[(0.1592457095160579, -0.8110438219448519, 0.5...","[0.2738683238526328, -0.8494990345766563, 0.45..."
5,"(4, 0, 10, 15, 14)",0,"[(0.8205581423833358, -0.09917465443647613, 0....","[0.8925515587120163, -0.0020453843824199225, 0..."
6,"(5, 6, 7, 8, 9)",0,"[(-0.1233863543090565, 0.08663485764000903, -0...","[5.334534269745011e-17, 5.615299231310538e-18,..."
7,"(5, 6, 16, 11, 17)",0,"[(-0.3478871122223193, 0.7497505146805842, -0....","[-0.28252234025388145, 0.8495807423184637, -0...."
8,"(6, 7, 15, 10, 16)",0,"[(0.6055520827818937, 0.5625459555918041, -0.5...","[0.7206950966718181, 0.5312296001674989, -0.44..."
9,"(7, 8, 19, 14, 15)",0,"[(0.7221388813398195, -0.40207799389106014, -0...","[0.7279364055224563, -0.5212627935849324, -0.4..."


In [253]:
#mypix.dfhexa

In [254]:

fig = plt.figure()
ax = Axes3D(fig)
ax.set_xlabel(r'$X$', fontsize=20)
ax.set_ylabel(r'$Y$', fontsize=20)
ax.set_zlabel(r'$Z$', fontsize=20)    


types = mypix.dfhexa.loc[:,'type'].values
center = np.stack(mypix.dfhexa.loc[:,'center'].values)
idx = mypix.dfhexa.loc[:,'idx'].values
idx=[i[0] for i in idx]
ax.scatter(center[:,0],center[:,1],center[:,2],marker='x',s=10)
#for k in range(center.shape[0]):
#    ax.text(center[k,0]*1.1,center[k,1]*1.1,center[k,2]*1.1,'%s' % (str(idx[k])), 
#                            size=10, zorder=1, color='k')


faces = np.stack(mypix.dfhexa.loc[:,'vertices'].values)
for f in range(faces.shape[0]):
    xf,yf,zf = faces[f,:,0],faces[f,:,1],faces[f,:,2]
    vertsf=list(zip(xf,yf,zf))
    if types[f]==1:      # hexagones inside icosaedre face
        col='yellow'
    elif types[f]==2:  # hexagones edges between 2 icosaedre faces
        col='green'
    else:                  # hexagones edges between 2 icosaedre faces neighboors of pentagons
        col='red'
    ax.add_collection3d(Poly3DCollection([vertsf], facecolors = col, edgecolors='k', linewidths=1, alpha=0.9))

    
for row in mypix.dfpenta.itertuples():
    vertsf=row.vertices
    ax.add_collection3d(Poly3DCollection([vertsf], facecolors = 'purple', edgecolors='k', linewidths=1, alpha=1))
    xyzc = row.center
    ax.scatter(xyzc[0],xyzc[1],xyzc[2],marker='o',s=15,color='k')
    
ax.set_xlim([-1,1])
ax.set_ylim([-1,1])
ax.set_zlim([-1,1])

    
plt.show()

In [297]:
mypix.dfhexa

,idx,type,vertices,center,c2,norm
0,"(12, 1, 2)",1,"[[-0.992251, 0.0, -0.12425199999999999], [-0.9...","[-0.9989889999999999, 0.0, 0.044947999999999995]","[-0.40158227541210256, -1.8885270718791305e-18...",0.968970
1,"(12, 1, 3, 18, 0, 3)",3,"[[-0.9570599999999999, 0.0, -0.289889], [-0.99...","[-0.9592529999999999, -0.160622, -0.232454]","[-0.38522278650146313, -0.06535408258529712, -...",0.966837
2,"(12, 0, 3, 17, 3, 0)",3,"[[-0.985772, 0.159297, -0.053656999999999996],...","[-0.9592529999999999, 0.160622, -0.232454]","[-0.386328683841411, 0.06051144498325572, -0.0...",0.969367
3,"(12, 2, 2, 18, 0, 2)",2,"[[-0.985772, -0.159297, -0.053656999999999996]...","[-0.9510569999999999, -0.309017, 0.0]","[-0.3808744144791217, -0.12723674487922199, 0....",0.967700
4,"(12, 0, 2, 17, 2, 0)",2,"[[-0.9363809999999999, 0.300816, 0.180833], [-...","[-0.9510569999999999, 0.309017, 0.0]","[-0.38287293015168095, 0.12269335208723056, 0....",0.970005
...,...,...,...,...,...,...
145,"(15, 2, 1)",1,"[[0.9367709999999999, 0.020336999999999997, -0...","[0.951333, 0.165815, -0.259752]","[0.3825383043017629, 0.06581024751343016, -0.1...",0.970516
146,"(15, 1, 2)",1,"[[0.905531, -0.26689599999999997, -0.329818], ...","[0.95868, -0.12405999999999999, -0.25601199999...","[0.38454521123907376, -0.05243472355044102, -0...",0.968970
147,"(14, 3, 1, 15, 0, 1)",3,"[[0.9343629999999999, -0.30702599999999997, 0....","[0.9592529999999999, -0.160622, 0.232454]","[0.3847163660798029, -0.05981634338782864, 0.1...",0.969881
148,"(10, 0, 1, 15, 1, 0)",3,"[[0.9039999999999999, 0.144119, 0.402509], [0....","[0.9592529999999999, 0.160622, 0.232454]","[0.38454835762020195, 0.06244253463350051, 0.1...",0.970889


Calcul des rayons Rin/Rout sur les pixels.

In [300]:
dftmp = mypix.dfhexa.copy(deep=True)

In [301]:
dftmp

,idx,type,vertices,center
0,"(12, 1, 2)",1,"[[-0.992251, 0.0, -0.12425199999999999], [-0.9...","[-0.9989889999999999, 0.0, 0.044947999999999995]"
1,"(12, 1, 3, 18, 0, 3)",3,"[[-0.9570599999999999, 0.0, -0.289889], [-0.99...","[-0.9592529999999999, -0.160622, -0.232454]"
2,"(12, 0, 3, 17, 3, 0)",3,"[[-0.985772, 0.159297, -0.053656999999999996],...","[-0.9592529999999999, 0.160622, -0.232454]"
3,"(12, 2, 2, 18, 0, 2)",2,"[[-0.985772, -0.159297, -0.053656999999999996]...","[-0.9510569999999999, -0.309017, 0.0]"
4,"(12, 0, 2, 17, 2, 0)",2,"[[-0.9363809999999999, 0.300816, 0.180833], [-...","[-0.9510569999999999, 0.309017, 0.0]"
...,...,...,...,...
145,"(15, 2, 1)",1,"[[0.9367709999999999, 0.020336999999999997, -0...","[0.951333, 0.165815, -0.259752]"
146,"(15, 1, 2)",1,"[[0.905531, -0.26689599999999997, -0.329818], ...","[0.95868, -0.12405999999999999, -0.25601199999..."
147,"(14, 3, 1, 15, 0, 1)",3,"[[0.9343629999999999, -0.30702599999999997, 0....","[0.9592529999999999, -0.160622, 0.232454]"
148,"(10, 0, 1, 15, 1, 0)",3,"[[0.9039999999999999, 0.144119, 0.402509], [0....","[0.9592529999999999, 0.160622, 0.232454]"


In [302]:
dftmp['c2']=dftmp['vertices'].map(lambda x: np.mean(x,axis=0)).map(lambda x : x/np.sqrt(np.sum(x*x)))

In [306]:
dftmp['diff']=(dftmp['center']-dftmp['c2']).map(lambda x: np.mean(x))

In [309]:
np.max(np.abs(dftmp['diff'].values))

0.011163100363513137

In [315]:
dftmp=dftmp.drop('diff',axis=1)

In [316]:
dftmp

,idx,type,vertices,center,c2
0,"(12, 1, 2)",1,"[[-0.992251, 0.0, -0.12425199999999999], [-0.9...","[-0.9989889999999999, 0.0, 0.044947999999999995]","[-0.9992980543941387, -4.6994141528353916e-18,..."
1,"(12, 1, 3, 18, 0, 3)",3,"[[-0.9570599999999999, 0.0, -0.289889], [-0.99...","[-0.9592529999999999, -0.160622, -0.232454]","[-0.9596459102456248, -0.16280651163037313, -0..."
2,"(12, 0, 3, 17, 3, 0)",3,"[[-0.985772, 0.159297, -0.053656999999999996],...","[-0.9592529999999999, 0.160622, -0.232454]","[-0.9611440868234313, 0.15054594691879664, -0...."
3,"(12, 2, 2, 18, 0, 2)",2,"[[-0.985772, -0.159297, -0.053656999999999996]...","[-0.9510569999999999, -0.309017, 0.0]","[-0.9483904728826685, -0.316823898000824, 0.01..."
4,"(12, 0, 2, 17, 2, 0)",2,"[[-0.9363809999999999, 0.300816, 0.180833], [-...","[-0.9510569999999999, 0.309017, 0.0]","[-0.9522331312463729, 0.3051473887037462, 0.01..."
...,...,...,...,...,...
145,"(15, 2, 1)",1,"[[0.9367709999999999, 0.020336999999999997, -0...","[0.951333, 0.165815, -0.259752]","[0.951150257077915, 0.16363180663702193, -0.26..."
146,"(15, 1, 2)",1,"[[0.905531, -0.26689599999999997, -0.329818], ...","[0.95868, -0.12405999999999999, -0.25601199999...","[0.9569029543329929, -0.13047865480726722, -0...."
147,"(14, 3, 1, 15, 0, 1)",3,"[[0.9343629999999999, -0.30702599999999997, 0....","[0.9592529999999999, -0.160622, 0.232454]","[0.9568791273458572, -0.14877716548739897, 0.2..."
148,"(10, 0, 1, 15, 1, 0)",3,"[[0.9039999999999999, 0.144119, 0.402509], [0....","[0.9592529999999999, 0.160622, 0.232454]","[0.9559642950079553, 0.15522841904419105, 0.24..."


In [419]:
def radout(x,c):
    return np.max(np.sqrt(np.sum((x-c)**2,axis=1)))

In [420]:
dftmp['radout']=dftmp[['vertices','c2']].apply(lambda x: radout(x.vertices,x.c2), axis=1)

In [421]:
dftmp['radout'].describe()

count    150.000000
mean       0.186131
std        0.004525
min        0.180920
25%        0.183595
50%        0.184014
75%        0.187373
max        0.196474
Name: radout, dtype: float64

In [414]:
def altitude(a,b,c):
    """
        Heron formula c is the base of a triangle and (a,b) are the lengthes of the other edges.
    """
    p=(a+b+c)/2.
    return 2.*np.sqrt(p*(p-a)*(p-b)*(p-c))/c
    

In [433]:
def radint(x,c):
    # transform x:(x1,x2,...,xn) into (x1,x2,...,xn,x1)
    a = np.vstack((x,x[0]))
    # ||x2-x1||,||x3-x2||,...,||xn-xn-1||,||x1-xn||  (n terms)
    edgeLength = np.sqrt(np.sum(np.diff(a,axis=0)**2,axis=1))
    # ||c-x1||,||c-x2||,..., ||c-xn||      (n terms)
    radiiLength= np.sqrt(np.sum((x-c)**2,axis=1))    
    #
    alt=[]
    for i in range(len(radiiLength)-1):
        alt.append(altitude(radiiLength[i],radiiLength[i+1],edgeLength[i]))
    return np.min(alt)

In [434]:
x=dftmp['vertices'][0]

In [429]:
c=dftmp['c2'][0]

In [430]:
radint(x,c)

1:  [[-0.992251  0.       -0.124252]
 [-0.985772 -0.159297 -0.053657]
 [-0.980577 -0.155157  0.119982]
 [-0.977083  0.        0.212859]
 [-0.980577  0.155157  0.119982]
 [-0.985772  0.159297 -0.053657]
 [-0.992251  0.       -0.124252]]
2:  [0.1743593  0.17376602 0.1808647  0.1808647  0.17376602 0.1743593 ]
3:  [-9.99298054e-01 -4.69941415e-18  3.74619605e-02]
4:  [0.16186743 0.184014   0.17673067 0.17679828 0.17673067 0.184014  ]


0.14814957351492106

In [435]:
dftmp['radint']=dftmp[['vertices','c2']].apply(lambda x: radint(x.vertices,x.c2), axis=1)

In [437]:
dftmp['radint'].describe()

count    150.000000
mean       0.142389
std        0.003793
min        0.134265
25%        0.140334
50%        0.142607
75%        0.144990
max        0.148150
Name: radint, dtype: float64

In [438]:
dftmp['radout'].describe()

count    150.000000
mean       0.186131
std        0.004525
min        0.180920
25%        0.183595
50%        0.184014
75%        0.187373
max        0.196474
Name: radout, dtype: float64

In [452]:
dftmp['ratio_norm']=sqrt(3)/2*dftmp['radout']/dftmp['radint']

In [453]:
dftmp['ratio_norm'].describe()

count    12.000000
mean      1.135082
std       0.032677
min       1.068855
25%       1.137288
50%       1.137288
75%       1.159368
max       1.159368
Name: ratio_norm, dtype: float64

les pentagones..

In [442]:
dftmp = mypix.dfpenta.copy(deep=True)

In [443]:
dftmp

,idx,type,vertices,center
0,"(0, 1, 2, 3, 4)",0,"[(0.1233863543090565, 0.08663485764000903, 0.9...","[-5.334534269745011e-17, 5.615299231310538e-18..."
1,"(0, 1, 11, 16, 10)",0,"[(0.3478871122223193, 0.7497505146805842, 0.56...","[0.2777588761430904, 0.8482349175082626, 0.450..."
2,"(1, 2, 12, 17, 11)",0,"[(-0.6055520827818937, 0.5625459555918041, 0.5...","[-0.7208871325786141, 0.5262833938469894, 0.45..."
3,"(2, 3, 13, 18, 12)",0,"[(-0.7221388813398195, -0.40207799389106014, 0...","[-0.7232916261291257, -0.5229738923961758, 0.4..."
4,"(3, 4, 14, 19, 13)",0,"[(0.1592457095160579, -0.8110438219448519, 0.5...","[0.2738683238526328, -0.8494990345766563, 0.45..."
5,"(4, 0, 10, 15, 14)",0,"[(0.8205581423833358, -0.09917465443647613, 0....","[0.8925515587120163, -0.0020453843824199225, 0..."
6,"(5, 6, 7, 8, 9)",0,"[(-0.1233863543090565, 0.08663485764000903, -0...","[5.334534269745011e-17, 5.615299231310538e-18,..."
7,"(5, 6, 16, 11, 17)",0,"[(-0.3478871122223193, 0.7497505146805842, -0....","[-0.28252234025388145, 0.8495807423184637, -0...."
8,"(6, 7, 15, 10, 16)",0,"[(0.6055520827818937, 0.5625459555918041, -0.5...","[0.7206950966718181, 0.5312296001674989, -0.44..."
9,"(7, 8, 19, 14, 15)",0,"[(0.7221388813398195, -0.40207799389106014, -0...","[0.7279364055224563, -0.5212627935849324, -0.4..."


In [444]:
dftmp['c2']=dftmp['vertices'].map(lambda x: np.mean(x,axis=0)).map(lambda x : x/np.sqrt(np.sum(x*x)))

In [445]:
dftmp

,idx,type,vertices,center,c2
0,"(0, 1, 2, 3, 4)",0,"[(0.1233863543090565, 0.08663485764000903, 0.9...","[-5.334534269745011e-17, 5.615299231310538e-18...","[-5.334534269745011e-17, 5.615299231310538e-18..."
1,"(0, 1, 11, 16, 10)",0,"[(0.3478871122223193, 0.7497505146805842, 0.56...","[0.2777588761430904, 0.8482349175082626, 0.450...","[0.2777588761430904, 0.8482349175082626, 0.450..."
2,"(1, 2, 12, 17, 11)",0,"[(-0.6055520827818937, 0.5625459555918041, 0.5...","[-0.7208871325786141, 0.5262833938469894, 0.45...","[-0.7208871325786141, 0.5262833938469894, 0.45..."
3,"(2, 3, 13, 18, 12)",0,"[(-0.7221388813398195, -0.40207799389106014, 0...","[-0.7232916261291257, -0.5229738923961758, 0.4...","[-0.7232916261291257, -0.5229738923961758, 0.4..."
4,"(3, 4, 14, 19, 13)",0,"[(0.1592457095160579, -0.8110438219448519, 0.5...","[0.2738683238526328, -0.8494990345766563, 0.45...","[0.2738683238526328, -0.8494990345766563, 0.45..."
5,"(4, 0, 10, 15, 14)",0,"[(0.8205581423833358, -0.09917465443647613, 0....","[0.8925515587120163, -0.0020453843824199225, 0...","[0.8925515587120163, -0.0020453843824199225, 0..."
6,"(5, 6, 7, 8, 9)",0,"[(-0.1233863543090565, 0.08663485764000903, -0...","[5.334534269745011e-17, 5.615299231310538e-18,...","[5.334534269745011e-17, 5.615299231310538e-18,..."
7,"(5, 6, 16, 11, 17)",0,"[(-0.3478871122223193, 0.7497505146805842, -0....","[-0.28252234025388145, 0.8495807423184637, -0....","[-0.28252234025388145, 0.8495807423184637, -0...."
8,"(6, 7, 15, 10, 16)",0,"[(0.6055520827818937, 0.5625459555918041, -0.5...","[0.7206950966718181, 0.5312296001674989, -0.44...","[0.7206950966718181, 0.5312296001674989, -0.44..."
9,"(7, 8, 19, 14, 15)",0,"[(0.7221388813398195, -0.40207799389106014, -0...","[0.7279364055224563, -0.5212627935849324, -0.4...","[0.7279364055224563, -0.5212627935849324, -0.4..."


In [446]:
dftmp['radout']=dftmp[['vertices','c2']].apply(lambda x: radout(x.vertices,x.c2), axis=1)

In [447]:
dftmp['radout'].describe()

count    12.000000
mean      0.163627
std       0.005945
min       0.151197
25%       0.164774
50%       0.164774
75%       0.167451
max       0.167451
Name: radout, dtype: float64

In [448]:
dftmp['radint']=dftmp[['vertices','c2']].apply(lambda x: radint(x.vertices,x.c2), axis=1)

In [449]:
dftmp['radint'].describe()

count    12.000000
mean      0.124832
std       0.002374
min       0.122505
25%       0.123083
50%       0.123083
75%       0.127511
max       0.127511
Name: radint, dtype: float64

In [455]:
dftmp['ratio_norm']=sqrt(25+10*sqrt(5))/sqrt(50+10*sqrt(5))*dftmp['radout']/dftmp['radint']

In [456]:
dftmp['ratio_norm'].describe()

count    12.000000
mean      1.060363
std       0.030526
min       0.998495
25%       1.062423
50%       1.062423
75%       1.083049
max       1.083049
Name: ratio_norm, dtype: float64

In [458]:
2*sqrt(3)* (0.142**2)

0.06985014496763767

In [460]:
25/sqrt(25+10*sqrt(5)) * (0.125**2)

0.05676113500041882

In [461]:
0.05676113500041882/0.06985014496763767

0.8126129878000522

In [462]:
(sqrt(25+10*sqrt(5))/4) / (3/2*sqrt(3))

0.6622120602431384

In [492]:
def radout(x,c):
    return np.max(np.sqrt(np.sum((x-c)**2,axis=1)))

def altitude(a,b,c):
    """
        Heron formula c is the base of a triangle and (a,b) are the lengthes of the other edges.
    """
    p=(a+b+c)/2.
    return 2.*np.sqrt(p*(p-a)*(p-b)*(p-c))/c

def radint(x,c):
    # transform x:(x1,x2,...,xn) into (x1,x2,...,xn,x1)
    a = np.vstack((x,x[0]))
    # ||x2-x1||,||x3-x2||,...,||xn-xn-1||,||x1-xn||  (n terms)
    edgeLength = np.sqrt(np.sum(np.diff(a,axis=0)**2,axis=1))
    # ||c-x1||,||c-x2||,..., ||c-xn||      (n terms)
    radiiLength= np.sqrt(np.sum((x-c)**2,axis=1))    
    #
    alt=[]
    for i in range(len(radiiLength)-1):
        alt.append(altitude(radiiLength[i],radiiLength[i+1],edgeLength[i]))
    return np.min(alt)

def process(df,Rin_norm=None,Rout_norm=None):
    df['c2']=df['vertices'].map(lambda x: np.mean(x,axis=0)).map(lambda x : x/np.sqrt(np.sum(x*x)))
    df['radint']=df[['vertices','c2']].apply(lambda x: radint(x.vertices,x.c2), axis=1)
    if Rin_norm is not None:
         df['radint'] =  df['radint']/Rin_norm
    df['radout']=df[['vertices','c2']].apply(lambda x: radout(x.vertices,x.c2), axis=1)
    if Rout_norm is not None:
        df['radout']=df['radout']/Rout_norm 
    df.drop('c2',axis=1)
    return df
    
def analyse(mypix):
    npix = 10*(mypix.n)**2 + 2
    Rin_norm = sqrt(2*pi/(sqrt(3)*npix))
    Rout_norm = sqrt(8*pi/(3*sqrt(3)*npix))
    print(f'npix={npix}, Rin_norm={Rin_norm}, Rout_norm={Rout_norm}')
    #hexagones
    print("Process hexagones")
    df=process(mypix.dfhexa.copy(),Rin_norm=Rin_norm,Rout_norm=Rout_norm)
    print('hexa : Rin\n',df['radint'].describe())
    print('hexa : Rout\n',df['radout'].describe())
    df.drop(['radint','radout'],axis=1)
    #pentagones
    print("Process pentagones")
    df=process(mypix.dfpenta.copy(),Rin_norm=Rin_norm,Rout_norm=Rout_norm)
    print('penta: Rin\n',df['radint'].describe())
    print('penta: Rout\n',df['radout'].describe())
    df.drop(['radint','radout'],axis=1)
    

In [493]:
analyse(mypix)

npix=100002, Rin_norm=0.006022894800629409, Rout_norm=0.006954639868888374
Process hexagones
hexa : Rin
 count    99990.000000
mean         0.935813
std          0.023591
min          0.880041
25%          0.916918
50%          0.931646
75%          0.952086
max          0.998735
Name: radint, dtype: float64
hexa : Rout
 count    99990.000000
mean         1.070504
std          0.064642
min          0.975077
25%          1.019259
50%          1.062875
75%          1.114826
max          1.237616
Name: radout, dtype: float64
Process pentagones
penta: Rin
 count    12.000000
mean      0.823859
std       0.019433
min       0.802507
25%       0.810637
50%       0.810637
75%       0.845623
max       0.845623
Name: radint, dtype: float64
penta: Rout
 count    12.000000
mean      0.941417
std       0.038927
min       0.859055
25%       0.951660
50%       0.951660
75%       0.964118
max       0.964118
Name: radout, dtype: float64


In [476]:
mypix=isocapix(100,fact=0)

makePentagon:  -0.5 0 100


In [494]:
mypix100fact0 = mypix

Test maintenant Goldberg modifiee mais d'abord Goldberg n=20 poiur reference 

In [499]:
mypix20fact0 =isocapix(20,fact=0)

makePentagon:  -0.5 0 20


In [500]:
analyse(mypix20fact0)

npix=4002, Rin_norm=0.030107249274679098, Rout_norm=0.03476485694659028
Process hexagones
hexa : Rin
 count    3990.000000
mean        0.936439
std         0.024758
min         0.880681
25%         0.918037
50%         0.934564
75%         0.952459
max         0.993344
Name: radint, dtype: float64
hexa : Rout
 count    3990.000000
mean        1.074354
std         0.059173
min         0.975917
25%         1.028680
50%         1.069813
75%         1.121084
max         1.211584
Name: radout, dtype: float64
Process pentagones
penta: Rin
 count    12.000000
mean      0.824758
std       0.018823
min       0.804393
25%       0.811796
50%       0.811796
75%       0.845866
max       0.845866
Name: radint, dtype: float64
penta: Rout
 count    12.000000
mean      0.941674
std       0.038178
min       0.861025
25%       0.951296
50%       0.951296
75%       0.964310
max       0.964310
Name: radout, dtype: float64


In [501]:
mypix20fact0d1 =isocapix(20,fact=0.1)

makePentagon:  -0.5 0 20


In [502]:
analyse(mypix20fact0d1)

npix=4002, Rin_norm=0.030107249274679098, Rout_norm=0.03476485694659028
Process hexagones
hexa : Rin
 count    3990.000000
mean        0.935715
std         0.026031
min         0.818906
25%         0.918037
50%         0.933594
75%         0.952087
max         0.993344
Name: radint, dtype: float64
hexa : Rout
 count    3990.000000
mean        1.074362
std         0.059160
min         0.975917
25%         1.028680
50%         1.069813
75%         1.121084
max         1.211584
Name: radout, dtype: float64
Process pentagones
penta: Rin
 count    12.000000
mean      0.920111
std       0.020897
min       0.897555
25%       0.905695
50%       0.905695
75%       0.943550
max       0.943550
Name: radint, dtype: float64
penta: Rout
 count    12.000000
mean      1.050411
std       0.042464
min       0.960731
25%       1.061045
50%       1.061045
75%       1.075648
max       1.075648
Name: radout, dtype: float64


In [503]:
mypix20fact0d05 =isocapix(20,fact=0.05)

makePentagon:  -0.5 0 20


In [504]:
analyse(mypix20fact0d05)

npix=4002, Rin_norm=0.030107249274679098, Rout_norm=0.03476485694659028
Process hexagones
hexa : Rin
 count    3990.000000
mean        0.936077
std         0.025215
min         0.849797
25%         0.918037
50%         0.934564
75%         0.952099
max         0.993344
Name: radint, dtype: float64
hexa : Rout
 count    3990.000000
mean        1.074357
std         0.059166
min         0.975917
25%         1.028680
50%         1.069813
75%         1.121084
max         1.211584
Name: radout, dtype: float64
Process pentagones
penta: Rin
 count    12.000000
mean      0.872431
std       0.019863
min       0.850967
25%       0.858741
50%       0.858741
75%       0.894708
max       0.894708
Name: radint, dtype: float64
penta: Rout
 count    12.000000
mean      0.996042
std       0.040324
min       0.910871
25%       1.006173
50%       1.006173
75%       1.019979
max       1.019979
Name: radout, dtype: float64


In [505]:
mypix20fact0d025 =isocapix(20,fact=0.025)
analyse(mypix20fact0d025)

makePentagon:  -0.5 0 20
npix=4002, Rin_norm=0.030107249274679098, Rout_norm=0.03476485694659028
Process hexagones
hexa : Rin
 count    3990.000000
mean        0.936258
std         0.024940
min         0.865232
25%         0.918037
50%         0.934564
75%         0.952397
max         0.993344
Name: radint, dtype: float64
hexa : Rout
 count    3990.000000
mean        1.074355
std         0.059170
min         0.975917
25%         1.028680
50%         1.069813
75%         1.121084
max         1.211584
Name: radout, dtype: float64
Process pentagones
penta: Rin
 count    12.000000
mean      0.848594
std       0.019343
min       0.827678
25%       0.835267
50%       0.835267
75%       0.870287
max       0.870287
Name: radint, dtype: float64
penta: Rout
 count    12.000000
mean      0.968858
std       0.039252
min       0.885946
25%       0.978735
50%       0.978735
75%       0.992145
max       0.992145
Name: radout, dtype: float64


In [507]:
%%time
mypix20fact0d04 =isocapix(20,fact=0.04)

makePentagon:  -0.5 0 20
CPU times: user 19.7 s, sys: 686 ms, total: 20.4 s
Wall time: 21.5 s


In [509]:
%%time
analyse(mypix20fact0d04)

npix=4002, Rin_norm=0.030107249274679098, Rout_norm=0.03476485694659028
Process hexagones
hexa : Rin
 count    3990.000000
mean        0.936149
std         0.025094
min         0.855968
25%         0.918037
50%         0.934564
75%         0.952099
max         0.993344
Name: radint, dtype: float64
hexa : Rout
 count    3990.000000
mean        1.074356
std         0.059168
min         0.975917
25%         1.028680
50%         1.069813
75%         1.121084
max         1.211584
Name: radout, dtype: float64
Process pentagones
penta: Rin
 count    12.000000
mean      0.862896
std       0.019655
min       0.841651
25%       0.849351
50%       0.849351
75%       0.884939
max       0.884939
Name: radint, dtype: float64
penta: Rout
 count    12.000000
mean      0.985168
std       0.039896
min       0.900900
25%       0.995198
50%       0.995198
75%       1.008846
max       1.008846
Name: radout, dtype: float64
CPU times: user 783 ms, sys: 13.9 ms, total: 797 ms
Wall time: 826 ms


In [510]:
%%time
mypix100fact0d05 =isocapix(100,fact=0.05)

makePentagon:  -0.5 0 100
CPU times: user 33min 26s, sys: 42.8 s, total: 34min 9s
Wall time: 34min 44s


In [511]:
%%time
analyse(mypix100fact0d05)

npix=100002, Rin_norm=0.006022894800629409, Rout_norm=0.006954639868888374
Process hexagones
hexa : Rin
 count    99990.000000
mean         0.935798
std          0.023612
min          0.849180
25%          0.916918
50%          0.931646
75%          0.952062
max          0.998735
Name: radint, dtype: float64
hexa : Rout
 count    99990.000000
mean         1.070504
std          0.064642
min          0.975077
25%          1.019259
50%          1.062875
75%          1.114826
max          1.237616
Name: radout, dtype: float64
Process pentagones
penta: Rin
 count    12.000000
mean      0.871435
std       0.020546
min       0.848865
25%       0.857454
50%       0.857454
75%       0.894445
max       0.894445
Name: radint, dtype: float64
penta: Rout
 count    12.000000
mean      0.995770
std       0.041163
min       0.908679
25%       1.006595
50%       1.006595
75%       1.019781
max       1.019781
Name: radout, dtype: float64
CPU times: user 17.1 s, sys: 167 ms, total: 17.3 s
Wall time: 17.4

In [515]:
mypix100fact0 = mypix20fact0

In [516]:
newpix20fact0 = isocapix(20,fact=0)

makePentagon:  -0.5 0 20


In [519]:
newpix5fact0 = isocapix(5,fact=0)

makePentagon:  -0.5 0 5


In [654]:
plotpix(newpix5fact0,face=1,alpha=0.5)

In [6]:
def plotpix(mypix,
              faces=None,
              plotcenter=False,
              plotidx=False,
              cols=['deepskyblue', 'yellow', 'lime','red'],
              linewidth=0.1,
              alpha=0.1
            ):

    """
    cols: 
     pentagones
     hexagones inside icosaedre face
     hexagones edges between 2 icosaedre faces
     hexagones edges between 2 icosaedre faces neighboors of pentagons
     ex. B&W setting    ['black', 'gray', 'silver','white']
        Colored setting ['deepskyblue', 'yellow', 'lime','red']
    """
    def extract(atype,idx,face):
        if (atype == 0) and not (face in idx): # penta
            return False
        elif (atype == 1) and (idx[0] != face): # hexa inside
            return False
        elif (atype > 1) and (idx[0] != face) and (idx[3] != face): #hexa edge
            return False
        else:
            return True
        return True

    df = mypix.dfAll
    if faces is not None:
        df = pd.DataFrame()
        for face in faces:
            df1 = mypix.dfAll[mypix.dfAll.apply(lambda x: extract(x.type,x.idx,face=face), axis=1)]
            df = pd.concat([df,df1], ignore_index=True)

    
    fig = plt.figure()
    ax = Axes3D(fig)
    ax.set_xlabel(r'$X$', fontsize=20)
    ax.set_ylabel(r'$Y$', fontsize=20)
    ax.set_zlabel(r'$Z$', fontsize=20)    
    
    for row in df.itertuples():
        
        atype  = row.type
        vertsf = row.vertices
        ax.add_collection3d(Poly3DCollection([vertsf], 
                                             facecolors = cols[atype], 
                                             edgecolors='k', 
                                             linewidths=linewidth, alpha=alpha))

        if plotcenter:
            xyzc = row.center
            ax.scatter(xyzc[0],xyzc[1],xyzc[2],marker='o',s=15,color='k')
        
        if (faces is not None) and plotidx:
            xyzc = row.center
            idx  = row.idx
            ax.text(xyzc[0],xyzc[1],xyzc[2],','.join(map(str, idx)), 
                                    size=7, zorder=10000, color='k')

    
    ax.set_xlim3d([-1,1])
    ax.set_ylim3d([-1,1])
    ax.set_zlim3d([-1,1])
    # Make panes transparent
    ax.xaxis.pane.fill = False # Left pane
    ax.yaxis.pane.fill = False # Right pane
    ax.zaxis.pane.fill = False
    # Remove grid lines
    ax.grid(False)


    plt.show()

In [676]:
plotpix(newpix5fact0,faces=[0,10],alpha=0.5,plotidx=True)

In [809]:
%timeit mapGridpoint2Sogere([0,0.5,0.5],[1,0,0],[0,1,0],[0,0,1])

19.2 µs ± 205 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [813]:
%timeit slerp([1,0,0],[0,1,0],0.1)

5.83 µs ± 192 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [796]:
%timeit barycentricCoords([1,2])

449 ns ± 6.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


11.9 ns ± 0.313 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [816]:
tmp = newpix5fact0.icoTriangs.copy()

In [817]:
tmp

array([[ 0,  1,  2],
       [ 0,  2,  3],
       [ 0,  3,  4],
       [ 0,  4,  5],
       [ 0,  5,  1],
       [ 6,  7,  8],
       [ 6,  8,  9],
       [ 6,  9, 10],
       [ 6, 10, 11],
       [ 6, 11,  7],
       [ 1,  2,  9],
       [ 2,  3,  8],
       [ 3,  4,  7],
       [ 4,  5, 11],
       [ 5,  1, 10],
       [ 1,  9, 10],
       [ 2,  8,  9],
       [ 3,  7,  8],
       [ 4, 11,  7],
       [ 5, 10, 11]])

In [819]:
tmp[10]

array([1, 2, 9])

In [821]:
tmp[10]=np.array([9,2,1])

In [822]:
tmp

array([[ 0,  1,  2],
       [ 0,  2,  3],
       [ 0,  3,  4],
       [ 0,  4,  5],
       [ 0,  5,  1],
       [ 6,  7,  8],
       [ 6,  8,  9],
       [ 6,  9, 10],
       [ 6, 10, 11],
       [ 6, 11,  7],
       [ 9,  2,  1],
       [ 2,  3,  8],
       [ 3,  4,  7],
       [ 4,  5, 11],
       [ 5,  1, 10],
       [ 1,  9, 10],
       [ 2,  8,  9],
       [ 3,  7,  8],
       [ 4, 11,  7],
       [ 5, 10, 11]])

In [7]:
mypix=isocapix(5,fact=0)

hexagon: 0.02 seconds
projFace: 0.10 seconds
projPt: 0.02 seconds
DfAppend: 0.00 seconds
Df processing All: 0.11 seconds
(verif) makeHexagon: 0.27 seconds
Hexa: 0.27 seconds
==> mapGridpoint2Sogere: 0.10 sec
==> slerp: 0.08 sec, cnt: 4560
==> barycentricCoords: 0.01 sec
Penta: 0.22 seconds


In [7]:
plotpix(mypix)

In [9]:
for id,ico in enumerate(mypix.icoTriangs):
    print(id,ico)

0 [0 1 2]
1 [0 2 3]
2 [0 3 4]
3 [0 4 5]
4 [0 5 1]
5 [6 7 8]
6 [6 8 9]
7 [ 6  9 10]
8 [ 6 10 11]
9 [ 6 11  7]
10 [1 2 9]
11 [2 3 8]
12 [3 4 7]
13 [ 4  5 11]
14 [ 5  1 10]
15 [ 1  9 10]
16 [2 8 9]
17 [3 7 8]
18 [ 4 11  7]
19 [ 5 10 11]
